Import relevant packages

In [1]:
import os
import pandas as pd
import numpy as np

Import movie data

In [2]:
df_movies = pd.read_csv('data/movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


one hot encode movie genres

In [3]:
# df_movies['genres'].str.split('|', expand = True).head()
df_movies = pd.concat([df_movies.drop(columns = 'genres'), df_movies['genres'].str.get_dummies()], axis = 1)
df_movies.head()

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


Decide that we will use Princess Bride, The (1987) as our reference movie to creat recomendations based on

In [4]:
movie='Princess Bride, The (1987)'
ind=int(df_movies.loc[df_movies['title'] == movie].index.values)
ind

899

In [5]:
movie_chosen = df_movies.loc[df_movies['title'] == 'Princess Bride, The (1987)', :].drop(columns = ['movieId', 'title']).transpose()[ind]
movie_chosen

(no genres listed)    0
Action                1
Adventure             1
Animation             0
Children              0
Comedy                1
Crime                 0
Documentary           0
Drama                 0
Fantasy               1
Film-Noir             0
Horror                0
IMAX                  0
Musical               0
Mystery               0
Romance               1
Sci-Fi                0
Thriller              0
War                   0
Western               0
Name: 899, dtype: int64

create a new data frame to store similarity scores

In [6]:
df_sim = df_movies[['movieId', 'title']].copy()
df_sim.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


Find all the movies similar to the above movie. First utilizing the pearson correlation.

In [7]:
## your code goes here
# create new column
df_sim['sim_pearson']=df_movies.corrwith(movie_chosen, axis=1, method='pearson')
# display top 5 rows
print('First 5 rows')
df_sim.head()

First 5 rows


,movieId,title,sim_pearson
0,1,Toy Story (1995),0.466667
1,2,Jumanji (1995),0.404226
2,3,Grumpier Old Men (1995),0.577350
3,4,Waiting to Exhale (1995),0.404226
4,5,Father of the Bride Part II (1995),0.397360


Now utilizing the jacard similarity

In [8]:
from sklearn.metrics import jaccard_score
## your code goes here
df_sim['sim_jaccard']=df_movies.corrwith(movie_chosen, axis=1, method=jaccard_score)
# display top 5 rows
print('First 5 rows')
df_sim.head()

First 5 rows


,movieId,title,sim_pearson,sim_jaccard
0,1,Toy Story (1995),0.466667,0.428571
1,2,Jumanji (1995),0.404226,0.333333
2,3,Grumpier Old Men (1995),0.577350,0.400000
3,4,Waiting to Exhale (1995),0.404226,0.333333
4,5,Father of the Bride Part II (1995),0.397360,0.200000


Now utilizing the cosine similarity by utilizing lambda instead of corrwith

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

## your code goes here
C_S= lambda X, Y : cosine_similarity(X=X.reshape(1,-1), Y=Y.reshape(1, -1))[0,0] # broke the lambda out for legibility
df_sim['sim_cosine']=df_movies.corrwith(movie_chosen, axis=1, method=C_S)
# display top 5 rows
print('First 5 rows')
df_sim.head()

First 5 rows


,movieId,title,sim_pearson,sim_jaccard,sim_cosine
0,1,Toy Story (1995),0.466667,0.428571,0.600000
1,2,Jumanji (1995),0.404226,0.333333,0.516398
2,3,Grumpier Old Men (1995),0.577350,0.400000,0.632456
3,4,Waiting to Exhale (1995),0.404226,0.333333,0.516398
4,5,Father of the Bride Part II (1995),0.397360,0.200000,0.447214


import movie ratings database to assist with the recommender

In [10]:
df_ratings = pd.read_csv('data/ratings.csv')
movie_user_mat = df_ratings.pivot_table(index = 'movieId', columns = 'userId', values = 'rating')
movie_user_mat.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


Extract the number of reviews and the average of the reviews per movie

In [11]:
## your code goes here
# because the index for movie_user_mat is set to movieId and not a 0 count we need to reset the movie_user_mat index
df_sim.set_index('movieId',inplace=True)
df_sim['ratings_avg']=movie_user_mat.mean(axis=1)
df_sim['ratings_cnt']=movie_user_mat.count(axis=1)
df_sim.head()

,title,sim_pearson,sim_jaccard,sim_cosine,ratings_avg,ratings_cnt
movieId,,,,,,
1,Toy Story (1995),0.466667,0.428571,0.600000,3.920930,215.0
2,Jumanji (1995),0.404226,0.333333,0.516398,3.431818,110.0
3,Grumpier Old Men (1995),0.577350,0.400000,0.632456,3.259615,52.0
4,Waiting to Exhale (1995),0.404226,0.333333,0.516398,2.357143,7.0
5,Father of the Bride Part II (1995),0.397360,0.200000,0.447214,3.071429,49.0


Find all movies that are in the top 40th percintile of cosine similarity of the princess bride and in top 20th percintile of ratings. Sort by highest rating.

In [12]:
## your code goes here
df_sim.loc[(df_sim['sim_cosine']>=df_sim['sim_cosine'].quantile(q=0.6))&(df_sim['ratings_avg']>=df_sim['ratings_avg'].quantile(q=0.8))][:5].sort_values(by='ratings_avg', axis=0, ascending=False,)

,title,sim_pearson,sim_jaccard,sim_cosine,ratings_avg,ratings_cnt
movieId,,,,,,
53,Lamerica (1994),0.192450,0.166667,0.316228,5.000000,2.0
28,Persuasion (1995),0.192450,0.166667,0.316228,4.227273,11.0
58,"Postman, The (Postino, Il) (1994)",0.404226,0.333333,0.516398,4.027027,37.0
29,"City of Lost Children, The (Cité des enfants p...",0.200000,0.250000,0.400000,4.013158,38.0
74,Bed of Roses (1996),0.192450,0.166667,0.316228,4.000000,8.0
